In [1]:
import sys
import numpy as np 
sys.path.append('../../../src/')
sys.path.append('../../../src/wikidata')
import matplotlib.pyplot as plt
from mask_dataset import MaskedDataset
dataset = MaskedDataset(model_name="mbert", reload=False)

Load pre-saved file /home/xzhao/workspace/probing-mulitlingual/result/dataset-mbert/mlama-dataset.pkl for MaskedDataset instance


In [ ]:
from utils import chunk_list_by_value_range
from wiki_evaluation import _get_correct_wrong_prediction_of_inwiki_fk

def plot_stacked_bar_and_percentage_of_inwiki_prediction(dataset, chunk_size=50, max_cnt=2500):
    raw_all_cnts, raw_corrt_cnts, raw_wrong_cnts = _get_correct_wrong_prediction_of_inwiki_fk(dataset)
    _, chunked_range, chunked_idx = chunk_list_by_value_range(raw_all_cnts[1:], chunk_size=chunk_size, max_val=max_cnt)
    x_labels = [f"{sidx + 1} ~ {eidx + 1}" for sidx, eidx in chunked_range]

    corrt_cnts = []
    wrong_cnts = []
    for idxs in chunked_idx:
        corrt_cnts.append(sum(raw_corrt_cnts[idx] for idx in idxs))
        wrong_cnts.append(sum(raw_wrong_cnts[idx] for idx in idxs))

    corrt_rate = [corrt_cnts[i]/(corrt_cnts[i] + wrong_cnts[i]) for i in range(len(corrt_cnts))]

    fig, ax1 = plt.subplots(figsize=(10, 7))
    ax1.bar(x_labels, corrt_cnts, label='in-wiki & predicted')
    ax1.bar(x_labels, wrong_cnts, bottom=corrt_cnts, label='in-wiki & non-predicted')
    ax1.legend()
    ax1.tick_params(axis='y')
    ax1.set_xticklabels(x_labels, rotation=90)
    
    
    ax2 = ax1.twinx()
    ax2.set_ylabel('The percentage of inwiki & predicted FK')
    ax2.plot(x_labels, corrt_rate, marker='o', linestyle='-')
    ax2.tick_params(axis='y')
    ax1.set_title('Distribution of wiki-occurred factual knowledge: predicted vs non-predicted')
    ax2.set_xlabel('The number of co-occurrences for subject-object')
    fig.tight_layout()
    plt.show()